In [1]:
#importamos módulos básicos
import os
# PIL de pillow es un buen módulo para procesar imágenes
import PIL
import PIL.Image
import numpy as np

In [2]:
# instalamos módulos para procesar rostros
!pip install -U insightface
!pip install onnxruntime
from insightface.app import FaceAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 56.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060438 sha256=7bcc4595d78ba59d25163d7db5f1097428d0606122c17a0fd4971e37f473d692
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [3]:
# descargar un pequeño dataset de prueba
!wget "https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=s6p3564g&dl=0" -O personas.zip

--2025-07-24 00:55:58--  https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=s6p3564g&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com/cd/0/inline/CuGGJIfpIGO_T-Yz-ELXW35ThGpBCkcMr7jxD5obG40l6axLioGryOm6LqcF9In1FoR_lzs0dpn65Ewi7w7_0-Q44ZlMAF49W0Ga4QumSJEXGsLvlA8S8_Y_gghqwWM4hTWwqx1O7zWf7f7F1nVif1ph/file# [following]
--2025-07-24 00:55:59--  https://uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com/cd/0/inline/CuGGJIfpIGO_T-Yz-ELXW35ThGpBCkcMr7jxD5obG40l6axLioGryOm6LqcF9In1FoR_lzs0dpn65Ewi7w7_0-Q44ZlMAF49W0Ga4QumSJEXGsLvlA8S8_Y_gghqwWM4hTWwqx1O7zWf7f7F1nVif1ph/file
Resolving uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com (uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com)... 162

In [4]:
# descomprimimos el dataset con 7 personalidades
!unzip personas.zip

Archive:  personas.zip
   creating: personas/
   creating: personas/barack/
  inflating: personas/barack/face_2.jpeg  
  inflating: personas/barack/barack.jpeg  
  inflating: personas/barack/barack_2.jpeg  
   creating: personas/dicaprio/
  inflating: personas/dicaprio/dicaprio_2.png  
  inflating: personas/dicaprio/dicaprio.jpg  
   creating: personas/pancho/
  inflating: personas/pancho/pancho.jpeg  
  inflating: personas/pancho/pancho2.jpeg  
   creating: personas/thalia/
  inflating: personas/thalia/thalia2.jpg  
  inflating: personas/thalia/thalia1.jpg  
   creating: personas/paulina/
  inflating: personas/paulina/paulina2.png  
  inflating: personas/paulina/paulina1.jpeg  
  inflating: personas/faces.txt      
   creating: personas/mario/
  inflating: personas/mario/mario1.png  
  inflating: personas/mario/mario2.png  
   creating: personas/michelle/
  inflating: personas/michelle/michelle_obama_2.jpg  
  inflating: personas/michelle/michelle_obama_1.jpeg  


In [21]:
datapath = '/content/personas'
ffaces = os.path.join(datapath, 'faces.txt')
fdata = []
with open(ffaces) as f:
  for line in f :
    iname, icl = line.split()
    iname = os.path.join(datapath, iname)
    fdata.append({'path': iname, 'name': icl})
print(fdata)

[{'path': '/content/personas/paulina/paulina2.png', 'name': 'paulina'}, {'path': '/content/personas/paulina/paulina1.jpeg', 'name': 'paulina'}, {'path': '/content/personas/dicaprio/dicaprio_2.png', 'name': 'dicaprio'}, {'path': '/content/personas/dicaprio/dicaprio.jpg', 'name': 'dicaprio'}, {'path': '/content/personas/barack/barack.jpeg', 'name': 'barack'}, {'path': '/content/personas/barack/barack_2.jpeg', 'name': 'barack'}, {'path': '/content/personas/barack/face_2.jpeg', 'name': 'barack'}, {'path': '/content/personas/pancho/pancho.jpeg', 'name': 'pancho'}, {'path': '/content/personas/pancho/pancho2.jpeg', 'name': 'pancho'}, {'path': '/content/personas/thalia/thalia1.jpg', 'name': 'thalia'}, {'path': '/content/personas/thalia/thalia2.jpg', 'name': 'thalia'}, {'path': '/content/personas/mario/mario1.png', 'name': 'mario'}, {'path': '/content/personas/mario/mario2.png', 'name': 'mario'}, {'path': '/content/personas/michelle/michelle_obama_1.jpeg', 'name': 'michelle'}, {'path': '/conten

In [15]:
# inicializamos el modelo de rostros
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])  # Use 'CUDAExecutionProvider' for GPU
app.prepare(ctx_id=0)  # ctx_id=-1 for CPU, 0 for GPU

def get_face_embedding(image_path):
    """Extract face embedding from an image"""
    img = PIL.Image.open(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    img = np.array(img)
    if img.shape[2] == 4 :
      img = img[:,:,:-1]
    faces = app.get(img)
    if len(faces) < 1:
        raise ValueError("No faces detected in the image")
    if len(faces) > 1:
        print("Warning: Multiple faces detected. Using first detected face")
    # retorna el primer rostro junto con su embedding
    return faces[0].embedding

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [30]:
# recorremos la lista de rostros
embs = []
names = []
compute = True
for person in fdata:
    emb = get_face_embedding(person['path'])
    person['emb'] = emb

In [25]:
# formar una matriz de embeddings
n = len(fdata)
dim = len(fdata[0]['emb'])
embs = np.zeros((n, dim), dtype = np.float32)
for i, person in enumerate(fdata) :
  embs[i, :] = person['emb']

In [26]:
print(embs.shape)

(15, 512)


In [27]:
#np.save("face_embs.npy", embs)
# embs = np.load("face_embs.npy")
#similarity search
embs_norm = embs / np.linalg.norm(embs, ord = 2, axis = 1,  keepdims = True)
sim = embs_norm @ np.transpose(embs_norm)
print(sim)


[[0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999]
 [0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999]
 [0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999]
 [0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999]
 [0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999]
 [0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999999
  0.9999999]
 [0.9999999 0.9999999 0.9999999 0.9999999 0.9999999 0.9999

In [28]:
idx_sort =  np.argsort(-sim, axis = 1)
print(idx_sort)


[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]]


In [29]:
for  idx, row in enumerate(idx_sort) :
    name_query = names[idx]
    print('Query Name = {}'.format(name_query))
    result = [(names[i], sim[idx, i]) for i in row[1:5] ]
    print(result)


IndexError: list index out of range